<a href="https://colab.research.google.com/github/ChurikovSV/RAG_GigaChain/blob/main/RAG_v2_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gigachain gigachain-community unstructured[all-docs] faiss-cpu sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Создание фрагментов на основе PDF

In [9]:
from langchain_community.document_loaders import  PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                              chunk_overlap=100)
pdf_file = '/content/Шаблон_дз_по_курсу_Юнит_экономика_Беканова_Марианна.pdf'
loader =  PyPDFLoader(pdf_file)
splitted_data = loader.load_and_split(text_splitter)

Посмотрим сколько фрагментов получилось.

In [11]:
len(splitted_data)

23

Выведем на экран два соседних фрагмента, чтобы показать перекрытие.

In [12]:
splitted_data[10]

Document(metadata={'source': '/content/Шаблон_дз_по_курсу_Юнит_экономика_Беканова_Марианна.pdf', 'page': 1}, page_content='Referal R Показатель реферальных клиентов. То есть, пользователей, которые пришли к вам по рекомендации других пользователей.Необязательная метрика, которая покажет удовлетворённость нашими услугами.\nNet Promoter Score NPS Индекс лояльности пользователей. Так же как и referal, не несёт обязательного характера, но с его помощью можно оценить, насколько родители готовы порекомендовать нас своим друзьям и знакомым.\nСпецифичные метрики (если есть)\nКоэффициент приобретения дополнительных услуг в отношении к общему числу детей -\nПоказатель который получается путем количества покупок дополнительных курсов по отношению к общеу количеству детей на выбранный промежуток времениНеобходим для того, что бы можно было контролировать наиболее популярные среди родителей курсы и делать на них акцент при привлечении новых клиентов.')

In [13]:
splitted_data[11]

Document(metadata={'source': '/content/Шаблон_дз_по_курсу_Юнит_экономика_Беканова_Марианна.pdf', 'page': 1}, page_content='Custorer Satisfaction Indicator CSI Коэффициент удовлетворённости персоналом - метрика определяемая путём опроса родителей (возможно методом анкетирования). Необходим для понимания сильных и слабых сторон персонала.\nКоличество детей на одного воспитателя - Отношение количества детей, которые посещают детский сад, к количетву воспитателей и нянечек.Может быть важным фактором, влияющим на качество ухода за детьми и эффективность использования персонала.\nРождаемость - Рождаемость за определенный перид времениДаст понимание в актуальности предлагаемых услуг.')

Создание векторных представлений и векторного хранилища

В этом туториале опять используем модель paraphrase-multilingual-mpnet-base-v2 для векторизации фрагментов.

В качестве хранилища опять используем FAISS. Помимо этой векторной БД в GigaChain/LangChain есть большое количество интеграций с векторными хранилищами, как локальными так и облачными. Перечень доступных векторных хранилищ можно посмотреть здесь.

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

In [15]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(splitted_data, embedding=embedding)

<timed exec>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 23.5 s, sys: 6.41 s, total: 29.9 s
Wall time: 43.9 s


In [16]:
q1 = 'Насколько родители готовы порекомендовать нас своим друзьям и знакомым?'
q2 = 'Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?'
q3 = 'Имя какого английского писателя упоминается в статье?'

In [17]:
vector_store.similarity_search(q1)

[Document(metadata={'source': '/content/Шаблон_дз_по_курсу_Юнит_экономика_Беканова_Марианна.pdf', 'page': 0}, page_content='Открытие частного детского сада с фокусом на высоком уровне образования и заботы.\nИнтегрированная образовательная программа, развивающая интеллектуальные и социальные навыки детей.Гибкий график и индивидуальный подход к каждому ребенку. В случае необходимости индивидуальные занятия с логопедом, дефектологом, психологом, нейропсихологом и ABA-терапевтом.\nЛожная реклама: Конкурент может предоставлять ложные или вводящие в заблуждение сведения о своих услугах, чтобы привлечь родителей. Ценовой демпинг: Конкурент может временно снизить цены до нерентабельного уровня с целью временно привлечь больше клиентов и вытеснить других участников рынка.'),
 Document(metadata={'source': '/content/Шаблон_дз_по_курсу_Юнит_экономика_Беканова_Марианна.pdf', 'page': 0}, page_content='обучение.Семьи, предпочитающие обучение в рамках определенной культурной или религиозной обстановки

In [ ]:
vector_store.similarity_search(q2)

In [ ]:
vector_store.similarity_search(q3)


Создание фрагментов на основе DOCX



In [19]:
from langchain_community.document_loaders import PyWordDocumentLoader
loader = PyWordDocumentLoader('aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.docx')
splitted_docx = loader.load_and_split(text_splitter)

ImportError: cannot import name 'PyWordDocumentLoader' from 'langchain_community.document_loaders' (/usr/local/lib/python3.11/dist-packages/langchain_community/document_loaders/__init__.py)

In [ ]:
len(splitted_docx)

In [ ]:
splitted_data[10]

In [ ]:
splitted_docx[10]

In [ ]:
docs = embedding.embed_documents([splitted_data[10].page_content, splitted_docx[12].page_content])

In [ ]:
docs

In [ ]:
import numpy as np
cosine_similarity = np.dot(docs[0], docs[1]) / (np.linalg.norm(docs[0]) * np.linalg.norm(docs[1]))

print(f"Косинусное сходство: {cosine_similarity}")

Создание векторных представлений и векторного хранилища

In [ ]:
vector_store_docx = FAISS.from_documents(splitted_docx, embedding=embedding)

In [ ]:
vector_store_docx.similarity_search(q1)

Табличные данные

Теперь рассмотрим стандартные форматы табличных файлов: CSV и XLSX.

Для этого используем загрузчики CSVLoader и UnstructuredExcelLoader.

В качестве табличного файла используем документ "Динамика уровня и структуры затрат организаций на рабочую силу" с сайта Росстата.

Файл представляет собой несколько таблиц в основном с количественными данными. Поскольку файл изначально в формате XLSX, сделаем из него и документы в формате CSV. Так как в XLSX-файле несколько листов, придется создать отдельный CSV-файл под каждый лист

In [20]:
import pandas as pd

# Загрузите Excel файл
file_path = '/content/din-zatrat-rs.xlsx'

# Читаем все листы из Excel файла
sheets = pd.read_excel(file_path, sheet_name=None)

# Сохраняем каждый лист как отдельный CSV файл
for sheet_name, df in sheets.items():
    csv_file = f"{sheet_name}.csv"
    df.to_csv(csv_file, index=False)
    print(f"Сохранено: {csv_file}")

Сохранено: Содержание.csv
Сохранено: 1..csv
Сохранено: 2..csv
Сохранено: 3..csv
Сохранено: 4..csv
Сохранено: 5..csv
Сохранено: 6..csv
Сохранено: Основные понятия и определения.csv


In [21]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from glob import glob
files = glob('*.csv')

splitted_csv = []
for f in files:
  loader = CSVLoader(f)
  splitted_csv.extend(loader.load())

In [22]:
len(splitted_csv)

140

In [23]:
splitted_csv[139]

Document(metadata={'source': '5..csv', 'row': 25}, page_content='К содержанию: Оплата питания и проживания, всего\nUnnamed: 1: 0.5\nUnnamed: 2: 0.5\nUnnamed: 3: 0.2\nUnnamed: 4: 0.2\nUnnamed: 5: 0.6\nUnnamed: 6: 0.5')

Создание векторных представлений и векторного хранилища

In [24]:
vector_store_csv = FAISS.from_documents(splitted_csv, embedding=embedding)

In [25]:
q4 = 'Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?'
q5 = 'Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?'
q6 = 'В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?'

In [26]:
vector_store_csv.similarity_search(q4)

[Document(metadata={'source': 'Основные понятия и определения.csv', 'row': 2}, page_content='К содержанию: Затраты организаций на рабочую силу - это сумма вознаграждений работников (среднесписочной численности) в денежной и неденежной формах за отработанное и неотработанное время, расходы организации, связанные с обеспечением работников жильем, профессиональным обучением, культурно-бытовым обслуживанием, социальной защитой, включая расходы на пенсионное, медицинское и другие виды страхования, командировочные расходы, а также налоги и сборы, связанные с использованием наемной рабочей силы. В составе затрат на рабочую силу можно выделить две группы затрат: прямые и косвенные.\nПрямые затраты включают расходы, направленные организациями непосредственно на удовлетворение потребностей работников: заработная плата, расходы на жилье, выплаты социального характера, расходы на профессиональное обучение и культурно-бытовое обслуживание, оплата проезда и другие расходы, не отнесенные к приведенны

In [27]:
vector_store_csv.similarity_search(q5)

[Document(metadata={'source': 'Основные понятия и определения.csv', 'row': 2}, page_content='К содержанию: Затраты организаций на рабочую силу - это сумма вознаграждений работников (среднесписочной численности) в денежной и неденежной формах за отработанное и неотработанное время, расходы организации, связанные с обеспечением работников жильем, профессиональным обучением, культурно-бытовым обслуживанием, социальной защитой, включая расходы на пенсионное, медицинское и другие виды страхования, командировочные расходы, а также налоги и сборы, связанные с использованием наемной рабочей силы. В составе затрат на рабочую силу можно выделить две группы затрат: прямые и косвенные.\nПрямые затраты включают расходы, направленные организациями непосредственно на удовлетворение потребностей работников: заработная плата, расходы на жилье, выплаты социального характера, расходы на профессиональное обучение и культурно-бытовое обслуживание, оплата проезда и другие расходы, не отнесенные к приведенны

In [28]:
vector_store_csv.similarity_search(q6)

[Document(metadata={'source': 'Основные понятия и определения.csv', 'row': 2}, page_content='К содержанию: Затраты организаций на рабочую силу - это сумма вознаграждений работников (среднесписочной численности) в денежной и неденежной формах за отработанное и неотработанное время, расходы организации, связанные с обеспечением работников жильем, профессиональным обучением, культурно-бытовым обслуживанием, социальной защитой, включая расходы на пенсионное, медицинское и другие виды страхования, командировочные расходы, а также налоги и сборы, связанные с использованием наемной рабочей силы. В составе затрат на рабочую силу можно выделить две группы затрат: прямые и косвенные.\nПрямые затраты включают расходы, направленные организациями непосредственно на удовлетворение потребностей работников: заработная плата, расходы на жилье, выплаты социального характера, расходы на профессиональное обучение и культурно-бытовое обслуживание, оплата проезда и другие расходы, не отнесенные к приведенны

Загрузка табличных данных из xlsx-файла

In [40]:
!pip install nltk --quiet
import nltk

# Download the required data for sentence tokenization
nltk.download('punkt_tab')
# Download the averaged_perceptron_tagger for English
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

from langchain_community.document_loaders import UnstructuredExcelLoader

loader = UnstructuredExcelLoader('/content/din-zatrat-rs.xlsx')
splitted_xlsx = loader.load_and_split(text_splitter)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [41]:
len(splitted_xlsx)

28

In [42]:
splitted_xlsx[5]

Document(metadata={'source': '/content/din-zatrat-rs.xlsx'}, page_content='2013.0 2005 2007.0 2009.0 2013.0 Затраты на рабочую силу1) 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 100 100.0 100.0 100.0 в том числе: заработная плата 76.6 77.9 78.1 75.4 73.9 74.8 75.5 72.7 75.5 76.8 76.9 74.2 78.4 79.5 79.8 76.7 78.7 80.1 80.7 78.2 78.6 79.9 79.7 77.3 77.4 78.2 78.0 76.2 76.8 78.8 78.5 76.6 74.6 75.3 76.9 73.0 74.4 75.2 75.5 72.3 76.2 76.5 76.7 74.4 76.1 77.3 77.3 75.0 73.6 75.7 74.6 73.7 76.8 77.3 78.2 74.9 77.3 7

In [43]:
splitted_xlsx1 = loader.load_and_split()

In [44]:
len(splitted_xlsx1)

7

In [45]:
splitted_xlsx1[3]

Document(metadata={'source': '/content/din-zatrat-rs.xlsx'}, page_content='1) Затраты приведены в сопоставимой структуре без налогов и сборов, связанных с использованием рабочей силы. \nРазработка официальной статистической информации о составе затрат организаций на рабочую силу по видам экономической деятельности за 2005, 2007, 2009, 2013 гг. проводилась  с использованием Общероссийского классификатора видов экономической деятельности ОК 029-2007 (КДЕС Ред. 1.1).\n\nК содержанию\n\nСтруктура затрат на рабочую силу по Российской Федерации по видам экономической деятельности в 2017 и 2021 гг.\n\n(по организациям (без микропредприятий) обследуемых видов экономической деятельности; в процентах)\n\nВсего по обследованным видам экономической деятельности Добыча полезных ископаемых Обрабатывающие производства Обеспечение электрической энергией, газом и паром; кондиционирование воздуха Водоснабжение; водоотведение, организация сбора и утилизации отходов, деятельность по ликвидации загрязнений

In [46]:
loader = UnstructuredExcelLoader('din-zatrat-rs.xlsx', mode='elements')
splitted_xlsx2 = loader.load()

In [47]:
len(splitted_xlsx2)

43

In [48]:
splitted_xlsx2[5]

Document(metadata={'source': 'din-zatrat-rs.xlsx', 'filename': 'din-zatrat-rs.xlsx', 'last_modified': '2025-01-23T20:08:21', 'page_name': 'Содержание', 'page_number': 1, 'languages': ['rus'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'category': 'Title', 'element_id': 'aee66b36740a60fb5e6903643adb2d32'}, page_content='Шалаевская Татьяна Юрьевна')

In [49]:
vector_store_xlsx = FAISS.from_documents(splitted_xlsx1, embedding=embedding)

In [50]:
vector_store_xlsx.similarity_search(q4)

[Document(metadata={'source': '/content/din-zatrat-rs.xlsx'}, page_content='Всего по обследованным видам экономической деятельности Добыча полезных ископаемых Обрабатывающие производства из них: Производство и распределение электроэнергии, газа и воды Строительство Оптовая и розничная торговля; ремонт автотранспортных средств, мотоциклов, бытовых изделий и предметов личного пользования Гостиницы и рестораны Транспорт и связь из них: Финансовая деятельность Операции с недвижимым имуществом, аренда и предоставление услуг Сбор сточных вод, отходов и аналогичная деятельность производство пищевых продуктов, включая напитки, и табака текстильное и швейное производство производство кожи, изделий из кожи и производство обуви обработка древесины и производство изделий из дерева целлюлозно-бумажное производство, издательская и полиграфическая деятельность химическое производство металлургическое производство и производство готовых металлических изделий производство машин и оборудования производс